# How to build an advanced Chatbot with session memory using LangChain

**Objectives**

- Will be able to have a conversation.
- Will remember previous interactions: will have memory.
- Will be able to have different memories for different user sessions.
- Will be able to remember a limited number of messages: limited memory.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
# Connect with an LLM and start a conversation with it

from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
# mesage input

from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is blue."),
]

In [4]:
# call the LLM

chatbot.invoke(messagesToTheChatbot)

AIMessage(content='Blue is a calming and peaceful color that brings a sense of tranquility and serenity. It is also associated with trust, loyalty, and integrity. I love how versatile blue is, it can be bright and vibrant or soft and soothing. Blue reminds me of the sky and the ocean, both of which bring me a sense of peace and happiness. Overall, blue is a color that brings me joy and comfort, which is why it is my favourite.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 13, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3ca41e6-4005-4ae5-8730-1eaccf882290-0', usage_metadata={'input_tokens': 13, 'output_tokens': 91, 'total_tokens': 104})

In [5]:
# Check if the Chatbot remembers your favorite color.¶
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I do not have that information. Can you please tell me what your favorite color is?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 13, 'total_tokens': 35}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d8590bee-2c9a-44e4-b6e2-7fd551a77bdb-0', usage_metadata={'input_tokens': 13, 'output_tokens': 22, 'total_tokens': 35})

## Let's add memory to our Chatbot


In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

In [7]:
session1 = {"configurable": {"session_id": "001"}}

In [8]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is red.")],
    config=session1,
)

responseFromChatbot.content

"That's a bold and vibrant choice! Red is often associated with passion, energy, and strength. Do you have a favorite shade of red, like crimson or cherry red?"

In [9]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Based on your initial statement, your favorite color is red.'

In [10]:
## Let's now change the session_id and see what happens¶

In [11]:
session2 = {"configurable": {"session_id": "002"}}

In [12]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I'm sorry, I do not have enough information to determine your favorite color."

In [13]:
## Let's go back to session1 and see if the memory is still there

In [14]:
session1 = {"configurable": {"session_id": "001"}}

In [15]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red.'

In [16]:
## Our ChatBot has session memory now. Let's check if it remembers the conversation from session2.¶

In [17]:
session2 = {"configurable": {"session_id": "002"}}

In [18]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi name is Julio.")],
    config=session2,
)

responseFromChatbot.content

"Nice to meet you, Julio! If you'd like to share your favorite color, I'd be happy to know."

In [19]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)



In [20]:
responseFromChatbot.content

'Your name is Julio.'

In [21]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favorite color is red.'), AIMessage(content="That's a bold and vibrant choice! Red is often associated with passion, energy, and strength. Do you have a favorite shade of red, like crimson or cherry red?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 13, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ff7694a-c5ae-42ba-92e6-41b4b9f2b716-0', usage_metadata={'input_tokens': 13, 'output_tokens': 35, 'total_tokens': 48}), HumanMessage(content="What's my favorite color?"), AIMessage(content='Based on your initial statement, your favorite color is red.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 62, 'total_tokens': 74}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reaso

### Now, let's define a function to limit the number of messages stored in memory and add it to our chain with .assign.

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

In [23]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Vespa scooters.")],
    config=session1,
)

responseFromChatbot.content

'Vespa scooters are iconic and stylish vehicles! They have a classic design that is both practical and fun to ride. Do you have a favorite model or color of Vespa scooter?'

In [24]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is San Francisco.")],
    config=session1,
)

responseFromChatbot.content

'San Francisco is a beautiful and vibrant city known for its iconic landmarks, diverse culture, and picturesque views. What do you love most about San Francisco?'

In [27]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

In [25]:
# The chatbot memory has now 4 messages. Let's check the Chatbot with limited memory. 

In [28]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my favorite color?")],
    },
    config=session1,
)

responseFromChatbot.content

"I'm sorry, but as an AI assistant, I do not have personal information about you, including your favorite color. If you'd like to share your favorite color with me, I'd be happy to remember it for future reference."

In [29]:
# Finally, let's compare the previous response with the one provided by the Chatbot with unlimited memory

In [30]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="what is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Based on your previous statements, your favorite color is red.'